# Importando dependencias

In [ ]:
pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://git.neoway.io/api/v4/projects/273/packages/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyspark.sql.functions as sf
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql import DataFrame

In [ ]:
import gcsfs
import json
import pyarrow.parquet as pq

In [ ]:
from functools import reduce
import subprocess
import os

In [ ]:
from feature_store import FeatureStore, Catalog

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 150)

# Utils

In [ ]:
import gcsfs
import pyarrow.parquet as pq
def read_parquet_to_pandas(path: str, projectGCS="analytics-k8s-dev-4742"):
    """function to read parquet file as
     as pandas dataframe - useful for
     training tasks"""
    fs = gcsfs.GCSFileSystem(project=projectGCS, requester_pays=True)
    files = ["gs://" + path for path in fs.glob(path + "/part-*")]
    df_pandas = pq.ParquetDataset(files, filesystem=fs).read().to_pandas()
    return df_pandas

In [ ]:
BASE_GS = "gs://oculto/loss_prevention/perfil_laranja/data/"

In [ ]:
df = read_parquet_to_pandas("gs://oculto/loss_prevention/perfil_laranja/data/perfil_laranja_clean_baixaRenda_reduzido.parquet")

In [ ]:
df.shape

(4783994, 125)

# Modelagem

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_squared_error

In [ ]:
pfs_sem_patrimonio_cond = (
    (df["qtdImoveis"] == 0) &
    (df["qtdVeiculosFinanciados"] == 0) &
    (df["quantidadeVeiculosPesados"] == 0) &
    (df["quantidadePropriedades"] == 0) &
    (df["quantidadeAeronaves"] == 0)
)

df = df[~pfs_sem_patrimonio_cond]

In [ ]:
df.shape

(3411864, 125)

In [ ]:
df_sample = df.sample(frac=1, random_state=2025)

In [ ]:
X = df_sample.copy()
X.shape

(3411864, 125)

In [ ]:
1==1

True

In [ ]:
n_estimators = 100
contamination = 0.001
sample_size = 256

In [ ]:
iso_forest = IsolationForest(n_estimators=n_estimators,
                            contamination=contamination,
                            max_samples=sample_size,
                            random_state=42)

In [ ]:
%%time
iso_forest.fit(X)

CPU times: user 19.9 s, sys: 1.25 s, total: 21.1 s
Wall time: 21.1 s


IsolationForest(contamination=0.001, max_samples=256, random_state=42)

In [ ]:
df_sample['scoreAnomalia'] = iso_forest.decision_function(X)
df_sample['anomalia'] = iso_forest.predict(X)

In [ ]:
df_sample['anomalia'].value_counts()

anomalia
 1    3408452
-1       3412
Name: count, dtype: int64

In [ ]:
columns = X.columns

A importância é baseada na variação dos scores de anomalia ao embaralhar cada variável.

In [ ]:
%%time
original_scores = iso_forest.decision_function(X)


feature_importance = {}

for feature in X.columns:
    print(feature)
    df_permuted = X.copy()
    df_permuted[feature] = np.random.permutation(df_permuted[feature])

    permuted_scores = iso_forest.decision_function(df_permuted)
    score_change = mean_squared_error(original_scores, permuted_scores)  # Impacto no score de anomalia

    feature_importance[feature] = score_change

flagBeneficiarioBPC
flagVulneravelBPC
flagBeneficiarioSeguroDefeso
flagBeneficiarioGarantiaSafra
flagBeneficiarioNovoBolsaFamilia
flagBeneficiarioProgramasSociais
flagAltaQualificacaoBeneficiarioProgramasSociais
indicioFalecimento
flagFalecidoConfirmado
flagSupercentenario
flagAdolescente
flagIdoso
flagEmpresario
flagAltaQualificacao
flagServidorPublico
flagRegistroAntt
flagProdutorRural
quantidadeCandidatosApoiados
quantidadeEleicoesComoDoador
valorMedioDoado
valorTotalDoado
quantidadeDoacoesDescricaoNaoEspecificada
valorMaximoDoado
quantidadeDoacoesPoliticas
flagContratoPublicoBeneficiarioProgramasSociais
flagLaranja
quantidadeEmails
quantidadeDiferentesprovedoresEmail
quantidadeEmailPadraoSuspeito
quantidadeEmailNaoConsistenteNomeTitular
flagAreaUrbanaBaixaDensidadeEdificios
flagAglomeradoRural
flagAreaRural
flagFavelaOuComunidadeUrbana
flagDividaDauCresceu180Dias
valorTotalDividasDau
qtdVeiculosFinanciados
qtdFinanciamentosVeicularesQuitados
recorrenciaFinanciamentoVeicular
Recorre

In [ ]:
max_importance = max(feature_importance.values())
for feature in feature_importance:
    feature_importance[feature] = (feature_importance[feature] / max_importance) * 100


df_importance = pd.DataFrame(
    list(feature_importance.items()), columns=["Feature", "Importance"]
).sort_values(by="Importance", ascending=False)

df_importance["Importance"] = df_importance["Importance"].round(2)

In [ ]:
df_importance = pd.DataFrame(
    list(feature_importance.items()), columns=["Feature", "Importance"]
).sort_values(by="Importance", ascending=False)

In [ ]:
df_importance.head(60)

,Feature,Importance
30,flagAreaUrbanaBaixaDensidadeEdificios,100.000000
32,flagAreaRural,81.858578
109,flagSocioClasseEmpresaFachadaALTA,79.036537
34,flagDividaDauCresceu180Dias,77.803796
117,evolucaoPatrimonioSEM_INFORMACAO,69.294574
16,flagProdutorRural,66.841157
44,flagAlterouBancoDeclaracaoIrpfUltimos5Anos,66.054219
13,flagAltaQualificacao,59.254413
119,evolucaoPatrimonioDIMINUIU,55.794244
122,perfilInvestidorDIVERSIFICADO,53.904412


In [ ]:
importance_dict = df_importance.set_index("Feature")["Importance"].to_dict()

In [ ]:
fs = gcsfs.GCSFileSystem(project="seu-projeto-gcp")

path = "gs://oculto/loss_prevention/perfil_laranja/data/df_importance_baixa_renda.json"

with fs.open(path, 'w') as f:
    json.dump(importance_dict, f)


In [ ]:
spark.createDataFrame(df_importance).write.mode("overwrite").parquet(
    "gs://oculto/loss_prevention/perfil_laranja/data/df_importance_baixa_renda.parquet"
)
